In [0]:
import dspy
from pydantic import BaseModel, Field

lm = dspy.LM(
    "databricks/databricks-llama-4-maverick"
)
dspy.configure(lm=lm)

import mlflow
from mlflow.models import infer_signature

# 1. Point MLflow to Databricks and set your experiment
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Workspace/Users/kevin.romero@databricks.com/GenAI/Experiments/Closure Notes Generator")
mlflow.dspy.autolog(log_compiles=True, log_evals=True)

In [0]:
class ClosureNotes(BaseModel):
  situation: str = Field(description="What was the context? You can summarize from the request description ")
  task: str = Field(description="What was the task assigned to you? You can summarize from the scoping notes")
  action: str = Field(description="How many and what sessions were delivered?")
  result: str = Field(description="What were the results of the actions taken?")

class ASQ(dspy.Signature):
  """
  Description of customer engagement on one of the following areas:
  - Data Engineering
  - AI/BI
  - Cloud Infrastructure
  - Data Science
  - ML/GenAI
  """
  cast: str = dspy.InputField(desc="Context, Actions, Success Criteria, Timeline, Sessions, Action Items of customer case")
  status_notes: str = dspy.InputField(desc="List of sessions and actions taken with the customer")
  case_description: str = dspy.InputField(desc="Description of customer engagement")
  closure_notes: ClosureNotes = dspy.OutputField(desc="Summary of case description, execution, and end result")

In [0]:
cast = """
Context 📝

Customer has some models hosted outside Databricks
Customer wants to understand MLOps before migrating their models
Customer wants to feel confident on best practices
Scope (or Actions) 🔭

Present MLOps best practices
Present migration plan
Answer any related questions
Success Criteria 📈

Customer understands MLOps best practices
Customer has a clear migration plan
Customer does not have further questions
Timeline ⏱️

TBD
Sessions

1 internal sync
1 customer sync
1 follow up if necessary
Action Items

Ask SA for context on external models.
"""

status_notes = """
#closing
Sep 24, Kevin: Second customer sync. Covered DABs, MLOps Stacks, and provided next steps to get started.
Sep 23, Kevin: Asked SA if customer is still interested in another sync.
Sep 10, Kevin: First customer sync. Presented MLOps best practices + demo. Customer is also interested in DevOps side. They will review MLOps resources and want to have CI/CD session.
Sep 2, Kevin: Created Slack channel and asked SA for context.
"""

case_description = """
Situation: Customer is in the middle of an evaluation of moving a number of models for investment analytics from local environments to Databricks. They do not feel comfortable with MLflow yet and how to manage models and code through environments - guiding them on these topics will improve their confidence to move these models to databricks.


Ask: Run the standard MLflow + MLops STS playbook to build confidence in the customer to establish their approach to MLops on databricks.


Desired Outcome (What is the desired outcome of the engagement)


Please comment on the customer readiness and the timeline to start the customer engagement
"""

closure_notes_template = """
STAR Method - Situation, Task, Action, Result

------------------------------------------------------------------------
SITUATION - 
What was the context? You can summarize from the request description 
------------------------------------------------------------------------
{situation}

------------------------------------------------------------------------
TASK -
What was the task assigned to you? You can summarize from the scoping notes
------------------------------------------------------------------------
{task}


------------------------------------------------------------------------
ACTION -
How many and what sessions did you deliver?
------------------------------------------------------------------------
{action}

------------------------------------------------------------------------
RESULT - 
What were the results of the action you took?
------------------------------------------------------------------------
{result}
"""

expected_output = ClosureNotes(
  situation="""
  Customer has some models hosted outside Databricks
  Customer wants to understand MLOps before migrating their models
  Customer wants to feel confident on best practices
  """,
  task="""
  Present MLOps best practices
  Present migration plan
  Answer any related questions
  """,
  action="""
  1 internal sync
  2 customer syncs
  """,
  result="""
  Customer understands MLOps best practices
  Customer understands MLOps Stacks and DevOps best practices
  Customer received next steps
  Customer does not have further questions
  """
)

In [0]:
from dspy.teleprompt import MIPROv2

predictor = dspy.Predict(ASQ)

demos = [  # <2>
    dspy.Example(  # <3>
        cast=cast,
        case_description=case_description,
        status_notes=status_notes,
        closure_notes=expected_output
    ).with_inputs("cast", "case_description", "status_notes"),
    dspy.Example(  # <3>
        cast=cast,
        case_description=case_description,
        status_notes=status_notes,
        closure_notes=expected_output
    ).with_inputs("cast", "case_description", "status_notes")
]

def semantic_metric(example, pred, trace=None):
    # Use a minimal DSPy program to judge equivalence
    judge = dspy.ChainOfThought("cast, case_description, status_notes, prediction -> same_meaning: bool")
    
    result = judge(
        cast=example.cast,
        case_description=example.case_description,
        status_notes=example.status_notes,
        prediction=pred
    )
    
    # Return 1.0 if same meaning, else 0.0
    return 1.0 if result.same_meaning else 0.0

# Use with demonstrations
teleprompter = MIPROv2(metric=semantic_metric, auto="medium")
closure_notes_generator_with_demos = teleprompter.compile(predictor, trainset=demos)

# Assume 'compiled_program' is your optimized DSPy module
with mlflow.start_run(run_name="Optimized_Model"):
    
    # 1. Define an example input/output for the signature
    input_example = {
      "cast": cast,
      "status_notes": status_notes,
      "case_description": case_description
      }
    prediction = closure_notes_generator_with_demos(cast=cast, status_notes=status_notes, case_description=case_description)
    signature = infer_signature(input_example, prediction.closure_notes)
    
    # 2. Log the DSPy model
    # Note: MLflow has a native dspy flavor in newer versions
    mlflow.dspy.log_model(
        dspy_model=closure_notes_generator_with_demos,
        artifact_path="model",
        input_example=input_example,
        signature=signature,
        registered_model_name="users.kevin_romero.closure_notes_generator"
    )

result = closure_notes_generator_with_demos(cast=cast, status_notes=status_notes, case_description=case_description)

print(closure_notes_template.format(**dict(result.closure_notes)))
print(result)